In [1]:
import pandas as pd
from util import *
from stationUtil import *
from graph import *

len(subway_graph) == len(interval_time) :  True


/home/tfk/miniconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SUM_FLUX_METRIC = pd.read_json("../database/SUM_FLUX_IO_METRIC.json")
IMPLICIT_METRIC = pd.read_json("../database/IMPLICIT_METRIC.json")

In [3]:
# 정규화함. 호선 별로
SUM_FLUX_METRIC_GRADE = SUM_FLUX_METRIC.copy(deep=True)
for line in (1, 2, 3, 4):
    SUM_FLUX_METRIC_GRADE.loc[line_num_to_range(line), "IN_FLUX":"OUT_FLUX"] = SUM_FLUX_METRIC_GRADE.loc[line_num_to_range(line), "IN_FLUX":"OUT_FLUX"].rank(method="min")
    SUM_FLUX_METRIC_GRADE.loc[line_num_to_range(line), "IN_FLUX":"OUT_FLUX"] /= SUM_FLUX_METRIC_GRADE.loc[line_num_to_range(line), "IN_FLUX":"OUT_FLUX"].max()
    SUM_FLUX_METRIC_GRADE.loc[line_num_to_range(line), "IN_FLUX":"OUT_FLUX"] *= 10
    
SUM_FLUX_METRIC_GRADE.loc[line_num_to_range(4), :].head()

,IN_FLUX,OUT_FLUX
401,10.000000,10.000000
402,3.571429,5.000000
403,6.428571,8.571429
404,2.142857,2.142857
405,5.714286,5.714286


In [4]:
# 정규화함. 호선 별로
IMPLICIT_METRIC_GRADE = IMPLICIT_METRIC.copy(deep=True)
for line in (1, 2, 3, 4):
    IMPLICIT_METRIC_GRADE.loc[line_num_to_range(line), "직장인구":"유동인구"] = IMPLICIT_METRIC_GRADE.loc[line_num_to_range(line), "직장인구":"유동인구"].rank(method="min")
    IMPLICIT_METRIC_GRADE.loc[line_num_to_range(line), "직장인구":"유동인구"] /= IMPLICIT_METRIC_GRADE.loc[line_num_to_range(line), "직장인구":"유동인구"].max()
    IMPLICIT_METRIC_GRADE.loc[line_num_to_range(line), "직장인구":"유동인구"] *= 10

IMPLICIT_METRIC_GRADE.loc[line_num_to_range(4), :].head()

,직장인구,주거인구,유동인구
401,6.428571,7.142857,7.857143
402,7.857143,8.571429,9.285714
403,10.000000,10.000000,10.000000
404,9.285714,9.285714,8.571429
405,8.571429,7.857143,7.142857


In [5]:
FLUX_METRIC_MEAN = SUM_FLUX_METRIC_GRADE.mean(axis=1)
IMPLICIT_METRIC_MEAN = IMPLICIT_METRIC_GRADE.mean(axis=1)

DF_METRIC = pd.concat([FLUX_METRIC_MEAN, IMPLICIT_METRIC_MEAN], axis=1)
DF_METRIC = DF_METRIC.rename(columns={0: "FLUX", 1: "IMPLICIT"})

DF_METRIC["LINE"] = DF_METRIC.index
DF_METRIC["LINE"] = DF_METRIC["LINE"].apply(get_line)
DF_METRIC.head()

,FLUX,IMPLICIT,LINE
95,4.500,0.750000,1
96,1.875,1.666667,1
97,3.000,3.250000,1
98,1.000,4.916667,1
99,1.500,5.000000,1


In [6]:
def calc_grade(_tup):
    flux, impl = _tup
    if flux < 2 and impl < 2:
        return 5
    elif flux < 4 and impl < 4:
        return 4
    elif flux < 6 and impl < 6:
        return 3
    elif flux < 8 and impl < 8:
        return 2
    elif flux <= 10 and impl <= 10:
        return 1

In [7]:
# Calculate GRADE
DF_METRIC["GRADE"] = (
    DF_METRIC["FLUX"].apply(str) + "," + DF_METRIC["IMPLICIT"].apply(str)
)
DF_METRIC["GRADE"] = DF_METRIC["GRADE"].apply(
    lambda _X: tuple(map(float, _X.split(",")))
)
DF_METRIC["GRADE"] = DF_METRIC["GRADE"].apply(calc_grade)

DF_METRIC.to_json("../database/METRIC_INTEGRATED.json")
DF_METRIC.head()

,FLUX,IMPLICIT,LINE,GRADE
95,4.500,0.750000,1,3
96,1.875,1.666667,1,5
97,3.000,3.250000,1,4
98,1.000,4.916667,1,3
99,1.500,5.000000,1,3
